In [15]:
'''
处理鼠标事件；
从而获得手写数字！
'''

import cv2;
import numpy as np
import time

# 创建一个空帧，定义(700, 700, 3)画图区域
frame = np.zeros((100, 100, 3), np.uint8) 

mousedown = False
last_measurement = current_measurement = np.array((0, 0), np.float32)

def OnMouseMove(event, x, y, flag, userdata):
    global frame, mousedown, current_measurement, last_measurement
    if event == cv2.EVENT_LBUTTONDOWN:
        mousedown = True
        last_measurement = np.array([[np.float32(x)], [np.float32(y)]]) # 当前测量
        current_measurement = np.array([[np.float32(x)], [np.float32(y)]]) # 当前测量
        #print('鼠标左键点击事件！')
        #print('x:%d,y:%d'%(x,y),mousedown)
        #cv2.line(frame, (0, 0), (100, 100), (255, 0, 0)) # 蓝色线为测量值     
        
        
    if event == cv2.EVENT_LBUTTONUP:
        mousedown = False
        #print('鼠标左键抬起事件！')
        #print('x:%d,y:%d'%(x,y),mousedown)
    
    if event == cv2.EVENT_MOUSEMOVE and mousedown is True:
        #print('鼠标移动事件！')
        #print('x:%d,y:%d'%(x,y))
        last_measurement = current_measurement # 把当前测量存储为上一次测量
        current_measurement = np.array([[np.float32(x)], [np.float32(y)]]) # 当前测量
        lmx, lmy = last_measurement[0], last_measurement[1] # 上一次测量坐标
        cmx, cmy = current_measurement[0], current_measurement[1] # 当前测量坐标
        #print('lmx:%f.1,lmy:%f.1,cmx:%f.1'%(lmx,lmy,cmx))
        cv2.line(frame, (lmx, lmy), (cmx, cmy), (255, 255, 255), thickness = 7) # 蓝色线为测量值        
        #print(str(event))
print('start!')
# 窗口初始化
cv2.namedWindow("kalman_tracker")
#opencv采用setMouseCallback函数处理鼠标事件，具体事件必须由回调（事件）函数的第一个参数来处理，该参数确定触发事件的类型（点击、移动等）
cv2.setMouseCallback("kalman_tracker", OnMouseMove)
while True:
    cv2.imshow("kalman_tracker", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        #break
        #timenow = str(int(time.time()))
        #cv2.imwrite('number' + timenow + '.jpg',frame)
        cv2.imwrite('number.jpg',frame)
        cv2.destroyWindow('kalman_tracker')
        break;
cv2.destroyAllWindows()

start!


In [ ]:
'''
边缘检测，裁剪
'''

#  -*- coding: utf-8 -*
import matplotlib.pyplot as plt
import pylab
import cv2
import numpy as np
from datetime import datetime
import os

t1 = datetime.now().timestamp()
#import the image
img = cv2.imread('number.jpg',1)
#转化为灰度图
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
(xh,yw) = img_gray.shape
#高斯平滑除噪
#img_blur = cv2.GaussianBlur(img,(5,5),0)
img_blur = cv2.medianBlur(img_gray,5)
#cv2.imshow('Gaussian',img_blur)
#canny算子 边缘检测

img_canny = cv2.Canny(img_blur,200,100)
#cv2.imshow('canny',img_canny)
#二值化处理
ret, img_bin = cv2.threshold(img_canny, 80, 255, cv2.THRESH_BINARY )
#cv2.imshow('bin',img_bin)
#print(img_bin)
#积分运算
imgI = cv2.integral(img_bin)

#分块
sat = np.arange(10*10).reshape(10,10)
(p,q) = sat.shape
w = int(yw / q)
h = int(xh / p)
if w <= 5:
    print('the image is too small to split!')
    os._exit(0)
#print(w,h)
p = range(p)
q = range(q)
sat[0][0] = imgI[h-1][w-1]

for n in q[1:]:
    sat[0][n] = imgI[h-1][w * (n+1) -1] - imgI[h-1][w * n -1]
for m in p[1:]:
    sat[m][0] = imgI[h * (m+1) - 1][w-1] - imgI[h * m - 1][w-1]
for m in p[1:]:
    for n in q[1:]:
        sat[m][n] = imgI[h * (m+1) - 1][w * (n+1) -1]  - imgI[h * (m+1) - 1][w * n -1] -imgI[h * m - 1][w * (n+1) -1] + imgI[h * m - 1][w * n -1] 

#print(sat)
#计算各块的能量密度
sat = sat / (w * h) 
#print(sat)
#选出能量密度较高的块
print('to draw:')
threshold1 = 10

#合并块
sated = np.ones((80,100))

def eightSearch(sated, m, n,mleft,ntop,mright,nbottom):
    #sat_search = [sat[m-1][n-1],sat[m-1][n],sat[m-1][n+1],sat[m][n-1],sat[m][n+1],sat[m+1][n-1],sat[m+1][n],sat[m+1][n+1]]
    sat_search = [(m-1,n-1),(m-1,n),(m-1,n+1),(m,n-1),(m,n+1),(m+1,n-1),(m+1,n),(m+1,n+1)]
    for sati in sat_search:
        s0 = sati[0]
        s1 = sati[1]
        
        if sated[s0][s1] != -1:
            left = s1 * w -1
            top = s0 * h -1
            right = s1 * w + w -1
            bottom = s0 * h + h -1
            sated[s0][s1] = -1
            if sat[s0][s1] > 10:
                #记录边界
                if left < mleft:
                    mleft = left
                elif right > mright:
                    mright = right
                if top < ntop:
                    ntop = top
                elif bottom > nbottom:
                    nbottom = bottom
                #循环
                sated,mleft,ntop,mright,nbottom = eightSearch(sated,s0,s1,mleft,ntop,mright,nbottom)
    return sated,mleft,ntop,mright,nbottom
  
for m in p[1:-2]:
    for n in q[1:-2]:
        if sated[m][n] != -1 and sat[m][n]> 10:
            sated[m][n] = -1
            sated,mleft,ntop,mright,nbottom = eightSearch(sated,m,n,n*w-1,m*h-1,n*w-1+w,m*h-1+h)            
#print('mleft:%d,ntop:%d,mright:%d,nbottom:%d'%(mleft,ntop,mright,nbottom))
subimg = img[ntop:nbottom, mleft:mright]
#cv2.rectangle(img,(mleft,ntop),(mright,nbottom),(0,0,255),1)
subimgshape = subimg.shape
addpad =(2*int(0.1 * subimgshape[0]),2*int(0.1 * subimgshape[1]))
#print(addpad)
frame = np.zeros((addpad[0] + subimgshape[0], addpad[1] + subimgshape[1], 3), np.uint8)
frame[int(addpad[0]/2):(int(addpad[0]/2)+subimgshape[0]),int(addpad[1]/2):(int(addpad[1]/2)+subimgshape[1])] = subimg
#print('sated',sated)
cv2.imshow('rect',img)
cv2.imshow('subimg',subimg)
cv2.imshow('frame',frame)
cv2.imwrite('subnumber.jpg',frame)
#height, width = frame.shape[:2]
res = cv2.resize(frame,(28, 28), interpolation = cv2.INTER_AREA )
cv2.imwrite('normalize.jpg',res)
t2 = datetime.now().timestamp()
print(t2-t1)
cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
'''
图片缩放：
'''
import cv2
import numpy as np
img = cv2.imread('subnumber.jpg', 0)
height, width = img.shape[:2]
res = cv2.resize(img,(32, 32), interpolation = cv2.INTER_AREA )
cv2.imshow('img',img)
cv2.imshow('res',res)
cv2.imwrite('normalize.jpg',res)
cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
'''
训练模型
'''
#%%
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r'D:\desktop\TensorFlow\cnn\MNIST_DATA/', one_hot=True)

print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape, mnist.validation.labels.shape)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import shutil

trained_dir = './checkpoint/'
if os.path.exists(trained_dir):
    shutil.rmtree(trained_dir)
os.makedirs(trained_dir)
checkpoint_prefix = os.path.join(trained_dir, 'model')

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])

#保存模型
saver = tf.train.Saver([W,b])
checkpoint_steps = 100

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

tf.global_variables_initializer().run()

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys})
    if (i + 1) % checkpoint_steps == 0:
        saver.save(sess, checkpoint_prefix, global_step=i+1)
        
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))


Extracting D:\desktop\TensorFlow\cnn\MNIST_DATA/train-images-idx3-ubyte.gz
Extracting D:\desktop\TensorFlow\cnn\MNIST_DATA/train-labels-idx1-ubyte.gz
Extracting D:\desktop\TensorFlow\cnn\MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting D:\desktop\TensorFlow\cnn\MNIST_DATA/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)
0.9185


In [1]:
print(sess.run(W))
print(sess.run(b))

NameError: name 'sess' is not defined

In [2]:
'''
训练模型
'''
#%%
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
'''
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(r'D:\desktop\TensorFlow\cnn\MNIST_DATA/', one_hot=True)

print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape, mnist.validation.labels.shape)
'''
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import shutil

trained_dir = './checkpoint/'
checkpoint_prefix = os.path.join(trained_dir, 'model-1000')
#W = tf.Variable(tf.zeros([784, 10]))
#b = tf.Variable(tf.zeros([10]))

sess = tf.InteractiveSession()
#保存模型
saver = tf.train.import_meta_graph("./checkpoint/model-1000.meta")

tf.global_variables_initializer().run()

sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(trained_dir)
if ckpt and ckpt.model_checkpoint_path:
    print('debug')
    print(ckpt.model_checkpoint_path)
    saver.restore(sess, tf.train.latest_checkpoint('./checkpoint/'))
    print(sess.run(W))
    print(sess.run(b))



debug
./checkpoint/model-1000
INFO:tensorflow:Restoring parameters from ./checkpoint/model-1000


NameError: name 'W' is not defined